# For casual understand Vietnamese

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
    Trainer
    , TrainingArguments
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
import os 
from datasets import load_dataset
import pandas as pd

os.environ['HUGGING_FACE_HUB_TOKEN'] = os.getenv("HUGGING_FACE_HUB_TOKEN")

/home/ltnga/ITDSIU21079/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Select model base
base_model = "meta-llama/Llama-3.2-3B-Instruct"
dataset_path = "VTSNLP/instruct_general_dataset"


In [3]:
model = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.22s/it]


In [4]:
config = LoraConfig(
    r=32,
    lora_alpha=32,   
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
)

In [5]:
model = get_peft_model(model, config)

In [6]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruct"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [7]:
from datasets import load_dataset
dataset = load_dataset("VTSNLP/instruct_general_dataset", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 4531804/4531804 [05:13<00:00, 14453.41 examples/s] 


In [8]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 4,
        num_train_epochs = 3,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        save_steps=1000,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "llama_v1",
        report_to = "none", 
    ))

/home/ltnga/ITDSIU21079/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 4531804/4531804 [20:22<00:00, 3708.23 examples/s] 


In [9]:
trainer.train()

Step,Training Loss
1,4.943300
2,5.090500
3,5.398700
4,5.006200
5,4.864300
6,5.079500
7,4.834000
8,4.321000
9,4.656200
10,4.517100


TrainOutput(global_step=100, training_loss=3.7845400834083556, metrics={'train_runtime': 166.1725, 'train_samples_per_second': 9.629, 'train_steps_per_second': 0.602, 'total_flos': 2.1219055799648256e+16, 'train_loss': 3.7845400834083556, 'epoch': 0.0003530599707666344})